In [1]:
import rssi
import numpy as np

interface = 'wlp1s0'
rssi_scanner = rssi.RSSI_Scan(interface)

In [2]:
from src import point, vector, line, source

In [3]:
sx, sy, sz = np.random.randint(0, 100, 1)[0], np.random.randint(0, 100, 1)[0],  np.random.randint(0, 100, 1)[0]

In [4]:
wifi = source.Source(sx, sy, sz)

In [5]:
initx, inity, initz = 0, 0, 0

In [6]:
start = point.Point(initx, inity, initz)

In [7]:
start.Intensity(wifi)

Intensity at x: 0, y: 0, z: 0, is I: 0.017271157167530225


0.017271157167530225

In [8]:
init_p1, init_p2 = point.Point(initx, inity, initz), point.Point(1, 1, initz)

In [9]:
init_line = line.Line(init_p1, init_p2)

In [10]:
interation = 100
found = False
counter = 0
step = 10

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_xy(init_line, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

Intensity at x: 0, y: 0, z: 0, is I: 0.017271157167530225
Moved to x: 7.0710678118654755, y: 7.071067811865475, z: 0
Intensity at x: 7.0710678118654755, y: 7.071067811865475, z: 0, is I: 0.019891275375052912
Moved to x: 14.142135623730951, y: 14.14213562373095, z: 0
Intensity at x: 14.142135623730951, y: 14.14213562373095, z: 0, is I: 0.022398124881067752
Moved to x: 21.213203435596427, y: 21.213203435596423, z: 0
Intensity at x: 21.213203435596427, y: 21.213203435596423, z: 0, is I: 0.024378416322176147
Moved to x: 28.284271247461902, y: 28.2842712474619, z: 0
Intensity at x: 28.284271247461902, y: 28.2842712474619, z: 0, is I: 0.025385098881370406
Moved to x: 35.35533905932738, y: 35.35533905932737, z: 0
Intensity at x: 35.35533905932738, y: 35.35533905932737, z: 0, is I: 0.02514680304562121
Moved to x: 32.99831645537222, y: 32.99831645537221, z: 0
Intensity at x: 32.99831645537222, y: 32.99831645537221, z: 0, is I: 0.025367941891633952
Moved to x: 30.64129385141706, y: 30.6412938514

In [11]:
new_direction = init_line.GetPerpendicularFromPoint(start, axis=2)

In [12]:
new_direction.slope_xy, new_direction.intercept_xy

(-1.0, 60.75880490195518)

In [13]:
start.Goto(initx, inity, initz)

Moved to x: 0, y: 0, z: 0


In [14]:
interation = 100
found = False
counter = 0
step = 10

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_xy(new_direction, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

Intensity at x: 0, y: 0, z: 0, is I: 0.017271157167530225
Moved to x: 7.0710678118654755, y: -7.071067811865475, z: 0
Intensity at x: 7.0710678118654755, y: -7.071067811865475, z: 0, is I: 0.01493406223858945
Moved to x: 4.714045207910317, y: -4.714045207910316, z: 0
Intensity at x: 4.714045207910317, y: -4.714045207910316, z: 0, is I: 0.015694040048474068
Moved to x: 2.3570226039551585, y: -2.3570226039551576, z: 0
Intensity at x: 2.3570226039551585, y: -2.3570226039551576, z: 0, is I: 0.016474975689661966
Moved to x: 0.0, y: 8.881784197001252e-16, z: 0
Intensity at x: 0.0, y: 8.881784197001252e-16, z: 0, is I: 0.017271157167530225
Moved to x: -2.3570226039551585, y: 2.3570226039551594, z: 0
Intensity at x: -2.3570226039551585, y: 2.3570226039551594, z: 0, is I: 0.018075303591578402
Moved to x: -4.714045207910317, y: 4.714045207910318, z: 0
Intensity at x: -4.714045207910317, y: 4.714045207910318, z: 0, is I: 0.01887845624648077
Moved to x: -7.0710678118654755, y: 7.071067811865476, z

In [15]:
new_direction_2 = new_direction.GetPerpendicularFromPoint(start, axis=2)

In [16]:
wifi_prediction = line.Line.IntersectLine(new_direction, new_direction_2, axis=2)

In [17]:
round(wifi_prediction.x, 0), round(wifi_prediction.y, 0)

(2.0, 59.0)

In [18]:
wifi.x, wifi.y, wifi.z

(2, 59, 48)

In [ ]:
init_p1, init_p3 = point.Point(initx, inity, initz), point.Point(0, 1, 1)

In [ ]:
init_line_2 = line.Line(init_p1, init_p3)

In [ ]:
interation = 100
found = False
counter = 0
step = 10

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_yz(init_line, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

In [ ]:
new_direction = init_line.GetPerpendicularFromPoint(start, axis=3)

In [ ]:
new_direction.slope_yz, new_direction.intercept_yz